# Brain Tumor Detection

### For Google Colab Users

In [ ]:
#@title ## Mount Your Google Drive & Map your Directory
#@markdown Please run this cell (`Ctrl+Enter` or `Shift+Enter`) and follow the steps printed bellow.
import os

from google.colab import drive
drive.mount('/content/gdrive')

# check Google Drive is mounted
if not os.path.isdir("/content/gdrive"):
  raise FileNotFoundError("Your Google Drive isn't mounted. Please run the above cell.")

assignment_dir = "/content/gdrive/MyDrive/workspace/BrainTumorDetection"  #@param{type:"string"}
assignment_dest = "/content/project"

# create symbolic link
!rm -f {assignment_dest}
!ln -s "{assignment_dir}" "{assignment_dest}"
print(f'Succesfully mapped (ln -s) "{assignment_dest}" -> "{assignment_dir}"')

# cd to linked dir
%cd -q {assignment_dest}
print(f'Succesfully changed directory (cd) to "{assignment_dest}"')

### Init

In [1]:
#@title ##Init

#@markdown Import required modules and observe your resources

# auto reload
%load_ext autoreload
%autoreload 2

# import
import torch
import torchvision
from PIL import Image
import matplotlib.pyplot as plt
import random
import os
import wandb
wandb.login()

from experiment import Experiment
from post_process import get_masks_from_heatmaps, present_masks, calc_iou
from data import DataGenerator, SegmentationGenerator
from train import get_best_models_dict

# Show resources
!nvidia-smi
# Show possible models
print("Possible models:")
models_dict = get_best_models_dict()
for model in sorted(models_dict):
    print("Model {model} {spaces}-> accuracy {acc}".format(spaces=" "*(25-len(model)), model=model, acc=models_dict[model]))

wandb: Currently logged in as: antebi-itai (use `wandb login --relogin` to force relogin)


Thu Jul 22 11:09:16 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro RTX 8000     Off  | 00000000:3A:00.0 Off |                  Off |
| 33%   30C    P8    13W / 260W |      0MiB / 48601MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Set Configs

In [2]:
default_config = {
    "model_name": "vgg19_seg",
    "input_size": (256,256), 
    "occlusion_size": (30,30),
    
    "epochs": 0,
  
    #"data_train_class": DataGenerator,         "data_train_path": os.path.join(".", "Data", "Training"),
    #"data_test_class": DataGenerator,          "data_test_path":  os.path.join(".", "Data", "Testing"),
    "data_train_class": SegmentationGenerator, "data_train_path": os.path.join(".", "SegmentationData", "train"), 
    "data_test_class": SegmentationGenerator,  "data_test_path":  os.path.join(".", "SegmentationData", "test-compact"),

    "gt_threshold": 0,
    "heatmap_threshold" : 0.80,

    "heat_layers" : [['features', layer_num] for layer_num in [0, 2, 5, 7, 10, 12, 14, 16, 19, 21, 23, 25, 28, 30, 32, 34]],
    "ref_heat_layer": ['classifier', -1],
    "mri_type": "T1c", 
    "train_batch_size": 100,
    "lr": 1e-4,
    "heatmap_batch_size": 256,
    "smallest_contour_len" : 30,
    "shuffle_data": True,
    "device": "cuda",
    "attributes_to_log": ["model_name", "occlusion_size", "heatmap_threshold"]
}

sweep_config = {
    'name' : 'my-sweep',
    'method' : 'random',
    'parameters' : {
        'occlusion_size' : {'values' : [(10,10), (20,20), (30,30), (40,40), (50,50), (60,60)]},
        'gt_threshold' : {'values' : [0, 1, 2]},
        'heatmap_threshold' : {
            'distribution' : 'uniform', 
            'min': 0.4, 
            'max': 0.9
        }
    },
    'metric': {
        'name': 'avg_iou',
        'goal' : 'maximize'
    }
}

def agent():
    with wandb.init(project="BrainTumorDetection", config=default_config):
        config = wandb.config
        experiment = Experiment(config)
        iou = experiment.run()


### Run Single Experiment

In [ ]:
agent()

### Sweep over many Experiments

In [ ]:
sweep_id = wandb.sweep(sweep_config, entity='UBRATS', project='Unsupervised-BRATS')
print("Sweep ID:", sweep_id)
count = 100
wandb.agent(sweep_id, function=agent, count=count)

Create sweep with ID: 83axjamh
Sweep URL: https://wandb.ai/ubrats/Unsupervised-BRATS/sweeps/83axjamh
Sweep ID: 83axjamh


wandb: Agent Starting Run: 3xti9xmv with config:
wandb: 	gt_threshold: 0
wandb: 	heatmap_threshold: 0.550324241541464
wandb: 	occlusion_size: [10, 10]
wandb: WARNING Ignored wandb.init() arg project when running a sweep
wandb: wandb version 0.11.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


loading vgg19_seg model...
loading best model state from: ./Models/vgg19_seg.pth


0it [00:00, ?it/s]

loading best model state from: ./Models/vgg19_seg.pth


Model's accuracy: 0.9


/home/labs/waic/itaian/.conda/envs/itai-env/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448238472/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Creating heatmap...


  6%|▋         | 16/256 [00:10<02:35,  1.54it/s]

### Debug

In [ ]:
from matplotlib import pyplot as plt

default_config["data_train_class"] = "data.SegmentationGenerator"
default_config["data_test_class"] = "data.SegmentationGenerator"

experiment = Experiment(default_config)

In [ ]:
def imshow(image, title):
    plt.figure()
    plt.imshow(image, cmap="gray")
    plt.title(title)
mins = []
maxs = []

for image_num in range(len(experiment.test_dataset) // 2):
    title = "#{image_num}".format(image_num=image_num)
    # Original image
    original_image, (gt_mask, gt_) = experiment.test_dataset[image_num * 2 + 1]
    mins.append(gt_mask.min().item())
    maxs.append(gt_mask.max().item())
    
    imshow(gt_mask, title)
    for i in range(5):
        imshow(gt_mask > i, title + " > " + str(i))
